## Prediction with Random Forest      
                                     
Topics covered:                     
   - Data cleaning & refactoring & feature engineering in a separate file            
   - Run random forest  
     - set mytr                      
     - autotune                       
   - Understanding RF output:        
     - variable importance plots:    
       - all, top 10 and permutation importance for categorical features      
     - partial dependence plot       
     - sub-sample analysis    
     - SHAP values
   - "Horse-race": model comparison  
     - OLS, Lasso, CART, RF and XGBoost model          
                                
Case studies:                  
  - CH16A Predicting apartment prices with random forest  
                                
Dataset:

    airbnb

In [2]:
import os
import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from mizani.formatters import percent_format
from patsy import dmatrices
from plotnine import *
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from statsmodels.tools.eval_measures import rmse

warnings.filterwarnings("ignore")


In [8]:
data = data.assign(
    n_bathrooms=lambda x: x["n_bathrooms"].fillna(np.median(x["n_bathrooms"].dropna())),
    n_beds=lambda x: np.where(x["n_beds"].isnull(), x["n_accommodates"], x["n_beds"]),
    f_bathroom=lambda x: x["f_bathroom"].fillna(1),
    f_minimum_nights=lambda x: x["f_minimum_nights"].fillna(1),
    f_number_of_reviews=lambda x: x["f_number_of_reviews"].fillna(1),
    ln_beds=lambda x: x["ln_beds"].fillna(0),
)


## PART I
### Loading and preparing data 
----------------------------------------------

In [7]:
data = pd.read_csv("data/airbnb_madrid_workfile_adj.csv")

In [9]:
data.isnull().sum().sum()

0

### Sample definition and preparation

We focus on normal apartments, n<8

In [10]:
data = data.loc[lambda x: x["n_accommodates"] < 8]

Copy a variable - purpose later, see at variable importance

In [11]:
data = data.assign(n_accommodates_copy=data.n_accommodates)

In [12]:
data.describe()

,usd_price_day,p_host_response_rate,n_accommodates,n_bathrooms,n_review_scores_rating,n_number_of_reviews,n_reviews_per_month,n_minimum_nights,n_beds,n_days_since,...,flag_review_scores_rating,flag_reviews_per_month,flag_n_number_of_reviews,ln_days_since,ln_days_since2,ln_days_since3,n_days_since2,n_days_since3,ln_review_scores_rating,n_accommodates_copy
count,20233.000000,20233.0,20233.000000,20233.000000,20233.000000,20233.000000,20233.000000,20233.000000,20233.000000,20233.000000,...,20233.000000,20233.000000,20233.0,20233.000000,20233.000000,20233.000000,2.023300e+04,2.023300e+04,20233.000000,20233.000000
mean,117.980626,0.0,3.019473,1.253398,4.669453,53.026442,1.940280,7.060940,1.829931,6.282932,...,0.170415,0.170415,0.0,6.282932,40.894425,273.354686,1.725053e+06,4.461300e+09,1.534179,3.019473
std,91.768908,0.0,1.521987,0.595112,0.427763,93.815632,1.826741,17.356968,1.243126,1.191328,...,0.376006,0.376006,0.0,1.191328,13.978358,131.588942,3.234207e+06,1.158312e+10,0.135768,1.521987
min,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,1.000000
25%,60.000000,0.0,2.000000,1.000000,4.610000,2.000000,0.730000,1.000000,1.000000,5.710427,...,0.000000,0.000000,0.0,5.710427,32.608977,186.211182,9.060100e+04,2.727090e+07,1.528228,2.000000
50%,100.000000,0.0,3.000000,1.000000,4.760000,15.000000,1.460000,2.000000,2.000000,6.388561,...,0.000000,0.000000,0.0,6.388561,40.813717,260.740936,3.528360e+05,2.095846e+08,1.560248,3.000000
75%,147.000000,0.0,4.000000,1.000000,4.880000,61.000000,2.540000,3.000000,2.000000,7.023759,...,0.000000,0.000000,0.0,7.023759,49.333190,346.504434,1.258884e+06,1.412468e+09,1.585145,4.000000
max,999.000000,0.0,7.000000,12.000000,5.000000,1092.000000,41.220000,364.000000,40.000000,8.546364,...,1.000000,1.000000,0.0,8.546364,73.040330,624.229217,2.649161e+07,1.363523e+11,1.609438,7.000000


In [13]:
data.price.describe()

count    20233.000000
mean       117.980626
std         91.768908
min          1.000000
25%         60.000000
50%        100.000000
75%        147.000000
max        999.000000
Name: price, dtype: float64

In [14]:
data.f_room_type.value_counts()

f_room_type
Entire          13805
Private room     5926
Shared room       260
Unknown           242
Name: count, dtype: int64

In [15]:
data.f_property_type.value_counts()

f_property_type
Apartment              18087
House                    995
Hotel/Serviced Stay      610
Hostel                   268
Traditional Stay         191
Unknown                   54
Alternative Stay          28
Name: count, dtype: int64

In [16]:
data.f_number_of_reviews.value_counts()

f_number_of_reviews
[1, 51)       11039
[51, 1092)     5745
[0, 1)         3448
99                1
Name: count, dtype: int64

### Create train and holdout samples

train is where we do it all, incl CV

In [17]:
data_train, data_holdout = train_test_split(data, train_size=0.7, random_state=42)


In [18]:
data_train.shape, data_holdout.shape


((14163, 227), (6070, 227))

Basic Variables inc neighbourhood

In [33]:
basic_vars = [
    "n_accommodates",
    "n_beds",
    "n_days_since",
    "f_property_type",
    "f_room_type",
    "f_bathroom",
    #"f_cancellation_policy",
    #"f_bed_type",
    "f_neighbourhood_cleansed",
]

Reviews

In [20]:
reviews = [
    "n_number_of_reviews",
    "flag_n_number_of_reviews",
    "n_review_scores_rating",
    "flag_review_scores_rating",
]


Dummy variables

In [21]:
amenities = [col for col in data if col.startswith("d_")]

Interactions for the LASSO from lecture 19

In [22]:
X1 = [
    "n_accommodates:f_property_type",
    "f_room_type:f_property_type",
    "f_room_type:d_familykidfriendly",
    "d_airconditioning:f_property_type",
    "d_cats:f_property_type",
    "d_dogs:f_property_type",
]
# with boroughs
X2 = [
    "f_property_type:f_neighbourhood_cleansed",
    "f_room_type:f_neighbourhood_cleansed",
    "n_accommodates:f_neighbourhood_cleansed",
]

In [35]:
predictors_1 = basic_vars
predictors_2 = basic_vars + reviews + amenities
predictors_E = basic_vars + reviews + amenities + X1 + X2

## PART II
### RANDOM FORESTS 
-------------------------------------------------------

Initialize RandomForestRegressor

In [24]:
rfr = RandomForestRegressor(
    random_state=42,
    criterion="squared_error",
    n_estimators=500,
    oob_score=True,
    n_jobs=-1
)

Define tune grid

In [25]:
tune_grid = {"max_features": [5, 7, 9], "min_samples_split": [6, 11]}

Add all of this to GridSearchCV which will iterate through all combinations of the tunegrid's parameter options

In [26]:
rf_random = GridSearchCV(
    rfr, tune_grid, cv=5, scoring="neg_root_mean_squared_error", verbose=3
)


One hot encode data

I GOT AN ERROR HERE! HELP!

In [38]:

predictors_2_cleaned
data_train.columns

Index(['f_room_type', 'f_property_type', 'f_room_type2',
       'f_neighbourhood_cleansed', 'usd_price_day', 'p_host_response_rate',
       'n_accommodates', 'n_bathrooms', 'n_review_scores_rating',
       'n_number_of_reviews',
       ...
       'flag_review_scores_rating', 'flag_reviews_per_month',
       'flag_n_number_of_reviews', 'ln_days_since', 'ln_days_since2',
       'ln_days_since3', 'n_days_since2', 'n_days_since3',
       'ln_review_scores_rating', 'n_accommodates_copy'],
      dtype='object', length=227)

In [36]:
import re
from patsy import dmatrices

# Clean column names to remove special characters
data_train.columns = data_train.columns.str.replace(r"[^\w\s]", "_", regex=True)

# Update predictors_2 to match cleaned column names
predictors_2_cleaned = [re.sub(r"[^\w\s]", "_", col) for col in predictors_2]

# Ensure the cleaned column names are in the dataframe
data_train = data_train.rename(columns=dict(zip(predictors_2, predictors_2_cleaned)))

# Create design matrices
y, X = dmatrices("price ~ " + " + ".join(predictors_2_cleaned), data_train)

Fit the model (**NOTE:** load fitted model below)

In [39]:
data_train.columns = data_train.columns.astype(str)
# Remove unwanted characters
data_train = data_train.rename(columns=lambda x: x.replace(".", "_"))
data_train = data_train.rename(columns=lambda x: x.replace("-", "_").replace(" ", "_"))
data_train = data_train.rename(columns=lambda x: f"var_{x}" if x[0].isdigit() else x)
data_train.columns = data_train.columns.str.replace(r"[\'\"]", "", regex=True)  # Remove apostrophes and quotes
data_train.columns = data_train.columns.str.replace(r"[^a-zA-Z0-9_]", "_", regex=True)  # Replace other special characters
data_train.columns = ["var_" + col if col[0].isdigit() else col for col in data_train.columns]  # Prefix numbers
data_train.columns = data_train.columns.str.replace(r"[^\w]", "_", regex=True)
print(data_train.columns)

Index(['f_room_type', 'f_property_type', 'f_room_type2',
       'f_neighbourhood_cleansed', 'usd_price_day', 'p_host_response_rate',
       'n_accommodates', 'n_bathrooms', 'n_review_scores_rating',
       'n_number_of_reviews',
       ...
       'flag_review_scores_rating', 'flag_reviews_per_month',
       'flag_n_number_of_reviews', 'ln_days_since', 'ln_days_since2',
       'ln_days_since3', 'n_days_since2', 'n_days_since3',
       'ln_review_scores_rating', 'n_accommodates_copy'],
      dtype='object', length=227)


In [40]:
print(data_train.columns.tolist())  # Show all column names


['f_room_type', 'f_property_type', 'f_room_type2', 'f_neighbourhood_cleansed', 'usd_price_day', 'p_host_response_rate', 'n_accommodates', 'n_bathrooms', 'n_review_scores_rating', 'n_number_of_reviews', 'n_reviews_per_month', 'n_minimum_nights', 'n_beds', 'n_days_since', 'd_unnamed_59', 'd_backyard', 'd_baking_sheet', 'd_books_and_reading_material', 'd_breakfast', 'd_carbon_monoxide_alarm', 'd_cleaning_products', 'd_dining_table', 'd_dishes_and_silverware', 'd_dryer', 'd_essentials', 'd_exercise_equipment', 'd_exterior_security_cameras_on_property', 'd_extra_pillows_and_blankets', 'd_fire_extinguisher', 'd_first_aid_kit', 'd_free_street_parking', 'd_freezer', 'd_hangers', 'd_heating', 'd_hockey_rink', 'd_host_greets_you', 'd_hot_tub', 'd_hot_water', 'd_hot_water_kettle', 'd_indoor_fireplace', 'd_iron', 'd_keypad', 'd_kitchen', 'd_lock_on_bedroom_door', 'd_long_term_stays_allowed', 'd_microwave', 'd_noise_decibel_monitors_on_property', 'd_pets_allowed', 'd_ping_pong_table', 'd_pool', 'd_

In [41]:
rf_model_1 = rf_random.fit(X, y.ravel())

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END max_features=5, min_samples_split=6;, score=-65.673 total time=   1.3s
[CV 2/5] END max_features=5, min_samples_split=6;, score=-66.374 total time=   1.3s
[CV 3/5] END max_features=5, min_samples_split=6;, score=-74.241 total time=   1.3s
[CV 4/5] END max_features=5, min_samples_split=6;, score=-70.403 total time=   1.3s
[CV 5/5] END max_features=5, min_samples_split=6;, score=-67.677 total time=   1.3s
[CV 1/5] END max_features=5, min_samples_split=11;, score=-66.825 total time=   1.3s
[CV 2/5] END max_features=5, min_samples_split=11;, score=-67.471 total time=   1.5s
[CV 3/5] END max_features=5, min_samples_split=11;, score=-75.585 total time=   1.4s
[CV 4/5] END max_features=5, min_samples_split=11;, score=-71.640 total time=   1.5s
[CV 5/5] END max_features=5, min_samples_split=11;, score=-69.287 total time=   1.5s
[CV 1/5] END max_features=7, min_samples_split=6;, score=-65.254 total time=   1.5s
[CV 2/5] EN

Save fitted model

In [ ]:
pickle.dump(rf_model_1, open("model_fits/random_forest.pkl", "wb"))

Load fitted model here

In [ ]:
rf_model_1 = pickle.load(open("model_fits/random_forest.pkl", "rb"))

Do all the above steps in one cell, for a bit complicate model

**NOTE:** This took around 3 minutes on a 2020 M1 Macbook Pro

In [ ]:
rfr = RandomForestRegressor(
    random_state=42,
    criterion="squared_error",
    n_estimators=500,
    oob_score=True,
    n_jobs=-1,
)

tune_grid = {
    "max_features": [8, 10, 12],
    "min_samples_split": [6, 11, 16],
}

rf_random = GridSearchCV(
    rfr, tune_grid, cv=5, scoring="neg_root_mean_squared_error", verbose=3
)

y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)

rf_model_2 = rf_random.fit(X, y.ravel())

Save fitted model

In [ ]:
pickle.dump(rf_model_2, open("model_fits/random_forest_broad.pkl", "wb"))

Load fitted model here

In [ ]:
rf_model_2 = pickle.load(open("model_fits/random_forest_broad.pkl", "rb"))

Evaluate models

Test set performance within the workout set

In [ ]:
model_performance = (
    pd.DataFrame(
        {
            "RMSE test": [
                rf_model_1.cv_results_["mean_test_score"].min(),
                rf_model_2.cv_results_["mean_test_score"].min(),
            ]
        },
        ["Model 1", "Model 2"],
    )
    * -1
)
model_performance

## PART III
### MODEL DIAGNOSTICS 
---

Extract data for model diagnostics from fitted GridSearchCV objects

In [ ]:
rf_model_2_var_imp_df = (
    pd.DataFrame(
        rf_model_2.best_estimator_.feature_importances_, X.design_info.column_names
    )
    .reset_index()
    .rename({"index": "varname", 0: "imp"}, axis=1)
    .assign(
        imp_percentage=lambda x: x["imp"] / x["imp"].sum(),
        varname=lambda x: x.varname.str.replace(
            "f_room_type[T.", "Room type:", regex=False
        )
        .str.replace("f_neighbourhood_cleansed[T.", "Borough:", regex=False)
        .str.replace("]", "", regex=False),
    )
    .sort_values(by=["imp"], ascending=False)
)


**1) full varimp plot, above a cutoff**

In [ ]:
cutoff = 0.013

rf_model_2_var_imp_plot = (
    ggplot(
        rf_model_2_var_imp_df.loc[lambda x: x.imp > cutoff],
        aes(x="reorder(varname, imp)", y="imp_percentage"),
    )
    + geom_point(color="blue", size=2.5)
    + geom_segment(
        aes(x="varname", xend="varname", y=0, yend="imp_percentage"),
        color="blue",
        size=2
    )
    + ylab("Importance (Percent)")
    + xlab("Variable Name")
    + coord_flip()
    + scale_y_continuous(labels=percent_format())
    + theme_bw()
)
rf_model_2_var_imp_plot


**2) full varimp plot, top 10 only**

In [ ]:
(
    ggplot(
        rf_model_2_var_imp_df.iloc[:10, :],
        aes(x="reorder(varname, imp)", y="imp_percentage"),
    )
    + geom_point(color="blue", size=2.5)
    + geom_segment(
        aes(x="varname", xend="varname", y=0, yend="imp_percentage"),
        color="blue",
        size=2,
    )
    + ylab("Importance (Percent)")
    + xlab("Variable Name")
    + coord_flip()
    + scale_y_continuous(labels=percent_format())
    + theme_bw()
)


**3) grouped variable importance - keep binaries created off factors together**

For this, you need to create an sklearn pipeline and put OneHotEncoding in it (before, encoding was done by patsy's dmatrices). This way permutation_importance can calculate factor variables' importance 

In [ ]:
categorical_columns = [col for col in predictors_2 if col.startswith("f_")]
numerical_columns = [col for col in predictors_2 if col not in categorical_columns]


In [ ]:
categorical_encoder = OneHotEncoder(handle_unknown="ignore")

preprocessing = ColumnTransformer(
    [
        ("cat", categorical_encoder, categorical_columns),
        ("num", "passthrough", numerical_columns),
    ]
)

rf_best_pipeline = Pipeline(
    [
        ("preprocess", preprocessing),
        ("regressor", rf_model_2.best_estimator_),  # put best model to pipeline
    ]
)


In [ ]:
rf_best_pipeline.fit(data_train[predictors_2], data_train.price)


This takes a while – So we saved the results, you can load it below

In [ ]:
result = permutation_importance(
    rf_best_pipeline,
    data_train[predictors_2],
    data_train["price"],
    n_repeats=10,
    random_state=45,
    n_jobs=-1,
)


Save results

In [ ]:
pickle.dump(result, open("model_fits/random_forest_permutation_importance.pkl", "wb"))

Load fitted model here

In [ ]:
result = pickle.load(open("model_fits/random_forest_permutation_importance.pkl", "rb"))

In [ ]:
grouped_imp = (
    pd.DataFrame(result.importances_mean, data_train[predictors_2].columns)
    .reset_index()
    .rename({"index": "varname", 0: "imp"}, axis=1)
    .assign(imp_percentage=lambda x: x["imp"] / x["imp"].sum())
    .sort_values(by=["imp"], ascending=False)
)


In [ ]:
(
    ggplot(
        grouped_imp.head(15),
        aes(x="reorder(varname, imp)", y="imp_percentage"),
    )
    + geom_point(color="blue", size=2.5)
    + geom_segment(
        aes(x="varname", xend="varname", y=0, yend="imp_percentage"),
        color="blue",
        size=2,
    )
    + ylab("Importance (Percent)")
    + xlab("Variable Name")
    + coord_flip()
    + scale_y_continuous(labels=percent_format())
    + theme_bw()
)


## Partial Dependence Plots 
-------------------------------------------------------


 Note: we do this on holdout set!

In [ ]:
plot_partial_dependence(
    rf_best_pipeline,
    data_holdout[predictors_2],
    ["n_accommodates"],
    feature_names=data_holdout[predictors_2].columns,
    line_kw={"marker": "o", "color": "blue"},
)
plt.grid()
plt.ylim(70, 130)
plt.show()


For categorical features, this is a bit complicated, first extract pdp values with `partial_dependence`, then do the plot

In [ ]:
roomtype_pdp = partial_dependence(
    rf_best_pipeline, data_holdout[predictors_2], ["f_room_type"], kind="average"
)

roomtype_pdp = (
    pd.DataFrame(roomtype_pdp["average"], columns=roomtype_pdp["values"][0].tolist())
    .T.reset_index()
    .rename({0: "Predicted price", "index": "Room type"}, axis=1)
)

(
    ggplot(roomtype_pdp, aes(x="Room type", y="Predicted price"))
    + geom_point(color="blue", size=2)
    + scale_y_continuous(limits=[60, 120], breaks=np.arange(60, 121, 10))
    + theme_bw()
)


### Subsample performance: RMSE / mean(y) 
---------------------------------------
NOTE  we do this on the holdout set.


Save predicted values on holdout set

In [ ]:
data_holdout_w_prediction = data_holdout.assign(
    predicted_price=rf_best_pipeline.predict(data_holdout[predictors_2])
)


Create nice summary table of heterogeneity

In [ ]:
def calculate_rmse(groupby_obj):
    return (
        groupby_obj.apply(
            lambda x: mean_squared_error(x.predicted_price, x.price, squared=False),
        )
        .to_frame(name="rmse")
        .assign(mean_price=groupby_obj.apply(lambda x: np.mean(x.price)).values)
        .assign(rmse_norm=lambda x: x.rmse / x.mean_price)
        .round(2)
    )


In [ ]:
# cheaper or more expensive flats - not used in book
grouped_object = data_holdout_w_prediction.assign(
    is_low_size=lambda x: np.where(x.n_accommodates <= 3, "small apt", "large apt")
).groupby("is_low_size")
accom_subset = calculate_rmse(grouped_object)


In [ ]:
grouped_object = data_holdout_w_prediction.loc[
    lambda x: x.f_neighbourhood_cleansed.isin(
        [
            "Westminster",
            "Camden",
            "Kensington and Chelsea",
            "Tower Hamlets",
            "Hackney",
            "Newham",
        ]
    )
].groupby("f_neighbourhood_cleansed")
neightbourhood_subset = calculate_rmse(grouped_object)


In [ ]:
grouped_object = data_holdout_w_prediction.loc[
    lambda x: x.f_property_type.isin(["Apartment", "House"])
].groupby("f_property_type")
proptype_subset = calculate_rmse(grouped_object)


In [ ]:
all_holdout = (
    pd.DataFrame(
        [
            mean_squared_error(
                data_holdout_w_prediction.price,
                data_holdout_w_prediction.predicted_price,
                squared=False,
            ),
            data_holdout_w_prediction.price.mean(),
        ],
        index=["rmse", "mean_price"],
    )
    .T.assign(rmse_norm=lambda x: x.rmse / x.mean_price)
    .round(2)
)
all_holdout.index = ["All"]


In [ ]:
type_rows = pd.DataFrame(
    None,
    index=["Apartment size", "Type", "Borough"],
    columns=["rmse", "mean_price", "rmse_norm"],
).fillna("")


### Table 16.2 Performance across subsamples

In [ ]:
pd.concat(
    [
        type_rows.iloc[[0]],
        accom_subset,
        type_rows.iloc[[1]],
        proptype_subset,
        type_rows.iloc[[2]],
        neightbourhood_subset,
        all_holdout,
    ]
)


## SHAP

In [ ]:
import shap

SHAP Explainer takes an encoded matrix as input

In [ ]:
rf_best_pipeline["preprocess"].fit(data_holdout.filter(predictors_2))

# transform categorical features
X_encoded = rf_best_pipeline["preprocess"].transform(
    data_holdout.filter(predictors_2)
)
new_feature_names = rf_best_pipeline["preprocess"].get_feature_names()
X_holdout = pd.DataFrame(X_encoded, columns=new_feature_names)

Calculate SHAP values for our best model

**NOTE:** Again, we do this on the holdout set!

Run time, around 50 minutes is on a 2020 M1 Macbook Pro with Monterery 12.6. To decrease run time, you can decrease the number of estimators (now 500) in the random forest object

In [ ]:
explainer = shap.Explainer(rf_best_pipeline["regressor"].predict, X_holdout)
shap_values = explainer(X_holdout)

Save SHAP values

In [ ]:
pickle.dump(explainer, open("model_fits/shap_values.pkl", "wb"))

Load SHAP values

In [ ]:
explainer = pickle.load(open("model_fits/shap_values.pkl", "rb"))

In [ ]:
shap.plots.beeswarm(shap_values, max_display = 15)

Can do the same with log scale

In [ ]:
shap.plots.beeswarm(shap_values, max_display = 15, log_scale = True)

Or in absolute values – see similarities with simple variable importance

In [ ]:
shap.plots.beeswarm(shap_values.abs, color="shap_red")


In [ ]:
shap.plots.bar(shap_values)

### Explanation for observation predicitons

Simple bar of SHAP values

In [ ]:
shap.plots.bar(shap_values[2])

#### Waterfall plot
The waterfall plot has the same information, represented in a different manner. Here we can see how the sum of all the SHAP values equals the difference between the prediction $f(x)$ and the expected value $E[f(x)]$.

In [ ]:
shap.plots.waterfall(shap_values[2])

Evaluate RF models on the holdout set

In [ ]:
_, X_holdout = dmatrices("price ~ " + " + ".join(predictors_2), data_holdout)

In [ ]:
holdout_performance = pd.DataFrame(
    {
        "Simple Random Forest": rmse(
            rf_model_1.predict(X_holdout), data_holdout["price"]
        ),
        "Broad Random Forest": rmse(
            rf_best_pipeline.predict(data_holdout[predictors_2]), data_holdout["price"]
        ),
    },
    index=["holdout RMSE"],
).T
holdout_performance

## PART IV
### HORSERACE: compare with other models 
-----------------------------------------------

1. OLS with dummies for area

 using model B

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [ ]:
y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)

ols_model = LinearRegression().fit(X, y)

holdout_performance.loc["OLS"] = rmse(
    ols_model.predict(X_holdout).ravel(), data_holdout["price"]
)
holdout_performance

Take a look at coefficients

In [ ]:
ols_model_coeffs_df = pd.DataFrame(
    ols_model.coef_.tolist()[0],
    index=X.design_info.column_names,
    columns=["ols_coefficient"],
).assign(ols_coefficient=lambda x: x.ols_coefficient.round(3))
ols_model_coeffs_df

2.  LASSO

using extended model w interactions

In [ ]:
from sklearn.linear_model import ElasticNet


The parameter l1_ratio corresponds to alpha in the glmnet R package while alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio = 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable, unless you supply your own sequence of alpha.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

In [ ]:
lasso_model = ElasticNet(l1_ratio=1, normalize=True, fit_intercept=True)


In [ ]:
lasso_model_cv = GridSearchCV(
    lasso_model,
    {"alpha": [i / 100 for i in range(1, 26, 1)]},
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3
)


In [ ]:
y, X = dmatrices("price ~ " + " + ".join(predictors_E), data_train)
_, X_holdout_lasso = dmatrices("price ~ " + " + ".join(predictors_E), data_holdout)

In [ ]:
lasso_model_cv.fit(X, y.ravel())


Holdout performance

In [ ]:
holdout_performance.loc["LASSO"] = rmse(
    lasso_model_cv.predict(X_holdout_lasso), data_holdout["price"]
)

holdout_performance

Non-zero coefficients

In [ ]:
pd.DataFrame(
    lasso_model_cv.best_estimator_.coef_.tolist(),
    index=X.design_info.column_names,
    columns=["lasso_coefficient"],
).assign(lasso_coefficient=lambda x: x.lasso_coefficient.round(3)).loc[
    lambda x: x.lasso_coefficient != 0
]


3. CART model

In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)


In [ ]:
cart_model = DecisionTreeClassifier(random_state=2018, criterion="gini")


Get potential ccp_alpha, regularization parameters

In [ ]:
path = cart_model.cost_complexity_pruning_path(X, y.ravel())
ccp_alphas, impurities = path.ccp_alphas, path.impurities


Apply random search to select a "best" alpha
 
    
 RandomizedSearchCV does not calculate all potential alphas, just a random subset


In [ ]:
cart_model_cv = RandomizedSearchCV(
    cart_model,
    {"ccp_alpha": ccp_alphas},
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3
)


cart_model_cv.fit(X, y.ravel())


In [ ]:
holdout_performance.loc["CART"] = rmse(
    cart_model_cv.predict(X_holdout), data_holdout["price"]
)

holdout_performance

4. XGBoost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgbm = XGBRegressor(learning_rate=0.1)

tune_grid = {"n_estimators": [i for i in range(200, 500, 50)], "max_depth": [1, 5, 10]}

xgbm_model_cv = GridSearchCV(
    xgbm, tune_grid, cv=5, scoring="neg_root_mean_squared_error", verbose=10, n_jobs=-1
)


In [ ]:
categorical_columns = [col for col in predictors_2 if col.startswith("f_")]
numerical_columns = [col for col in predictors_2 if col not in categorical_columns]

categorical_encoder = OneHotEncoder(handle_unknown="ignore")

preprocessing = ColumnTransformer(
    [
        ("cat", categorical_encoder, categorical_columns),
        ("num", "passthrough", numerical_columns),
    ]
)

xgbm_pipe = Pipeline(
    [("preprocess", preprocessing), ("regressor", xgbm_model_cv)], verbose=True
)


**NOTE:** This takes a while, you can import the fitted model below.

Run time (8.3 minutes) is on a 2020 M1 Macbook Pro with Monterery 12.6

In [ ]:
xgbm_pipe.fit(data_train[predictors_2], data_train.price)


In [ ]:
pickle.dump(xgbm_pipe,open('model_fits/xgb.pkl','wb'))

**NOTE:** Load the fitted model here

In [ ]:
xgbm_pipe = pickle.load(open("model_fits/xgb.pkl", "rb"))

In [ ]:
holdout_performance.loc["XGBoost"] = rmse(
    xgbm_pipe.predict(data_holdout[predictors_2]), data_holdout["price"]
)

holdout_performance

The next will be in final model, loads of tuning

In [ ]:
xgbm_broad = XGBRegressor()

In [ ]:
tune_grid = {
    "n_estimators": [i for i in range(50, 500, 50)],
    "max_depth": [1, 5, 10],
    "learning_rate": [0.02, 0.05, 0.1, 0.15, 0.2],
}

xgbm_model_cv_broad = GridSearchCV(
    xgbm_broad,
    tune_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=10,
    n_jobs=-1
)

In [ ]:
categorical_columns = [col for col in predictors_2 if col.startswith("f_")]
numerical_columns = [col for col in predictors_2 if col not in categorical_columns]

categorical_encoder = OneHotEncoder(handle_unknown="ignore")

preprocessing = ColumnTransformer(
    [
        ("cat", categorical_encoder, categorical_columns),
        ("num", "passthrough", numerical_columns),
    ]
)

xgbm_pipe_broad = Pipeline(
    [("preprocess", preprocessing), ("regressor", xgbm_model_cv_broad)], verbose=True
)


Run time (47.3 minutes) is on a 2020 M1 Macbook Pro with Monterery 12.6

In [ ]:
xgbm_pipe_broad.fit(data_train[predictors_2], data_train.price)


In [ ]:
pickle.dump(xgbm_pipe_broad, open("model_fits/xgb_broad.pkl", "wb"))

**NOTE:** Load the fitted model here

In [ ]:
xgbm_pipe_broad = pickle.load(open("model_fits/xgb_broad.pkl", "rb"))

In [ ]:
holdout_performance.loc["XGBoost broad"] = rmse(
    xgbm_pipe_broad.predict(data_holdout[predictors_2]), data_holdout["price"]
)

holdout_performance.round(3)

Our best performing model is the broadly tuned Random Forest

Interesting, that the broadly tuning model resulted in the same model as the simple tuning

In [ ]:
xgbm_pipe_broad["regressor"].best_params_

In [ ]:
xgbm_pipe["regressor"].best_params_

How to access all details on each hyperparameter combination (sorted by performance)

In [ ]:
pd.DataFrame(xgbm_pipe_broad["regressor"].cv_results_).sort_values(by=["rank_test_score"])